In [1]:
import scvi
import scanpy as sc
import os
import pandas as pd
import numpy as np
from cfp.external import CFJaxSCVI

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/optuna/study/_optimize.py:29: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
split = 5
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex"
adata_train = sc.read(os.path.join(output_dir, f"adata_train_{split}.h5ad"))
adata_test = sc.read(os.path.join(output_dir, f"adata_test_{split}.h5ad"))
adata_ood = sc.read(os.path.join(output_dir, f"adata_ood_{split}.h5ad")) 

In [ ]:
CFJaxSCVI.setup_anndata(adata_train)

n_latent=10
n_hidden=2048

vae = CFJaxSCVI(adata_train, gene_likelihood="normal", n_latent = n_latent, n_hidden = n_hidden)

vae.train(
    max_epochs=2000,
    batch_size=1024,
    plan_kwargs=dict(
        lr=1e-4,
    ),
    early_stopping=True,
    early_stopping_patience=20,
)

adata_ood.obsm["X_scVI"] = vae.get_latent_representation(adata_ood)
adata_ood.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_ood, give_mean="True")

In [ ]:
adata_ood.write(f"/lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/10_2048_{split}_reconstruction.h5ad")

In [ ]:
vae.save(f"/lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/10_2048_{split}")